In [96]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chisquare, norm
from sklearn.linear_model import LinearRegression

This notebook explore gaussian noise adjustments to linear regression

Lets first generate some mock data with gaussian noise

In [97]:
np.random.seed(42)  # For reproducibility

In [98]:
actual_slope = 2
actual_intercept = 5
n_samples = 100  # Number of data points

x = np.random.rand(n_samples, 1) * 10  # Generate random x values between 0 and 10

noise_scale = 3

noise = noise_scale * np.random.randn(n_samples, 1)  # Gaussian noise

you can adjust these accordingly

In [99]:
y = actual_slope * x + actual_intercept + noise  # Linear function with Gaussian noise

Perform linear regression, we use scikit learn for ease

In [100]:
model = LinearRegression()
model.fit(x, y)  

LinearRegression()

Just now get the coefficients for reference.

In [101]:
slope = model.coef_[0]  # The slope 'a'
intercept = model.intercept_  # The intercept 'b'

print(f"Slope (a): {slope}")
print(f"Intercept (b): {intercept}")

Slope (a): [1.86206803]
Intercept (b): [5.64528847]


The residues need to be calculated

In [102]:
y_pred = model.predict(x)
residuals = y - y_pred
residuals = residuals.reshape(100)

after this normally we just show the MSE against the test data. but we're going to use the estimates and our knowledge of the gaussian noise to try and improve the estimates

lets use a simple chi squared model to fit the residues into a normal distribution

In [103]:
num_bins = 10  
# Example: 10 bins for the residuals, you can edit this as you like
hist, bin_edges = np.histogram(residuals, bins=num_bins, density=False)
# Compute bin centers
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])  
mean = np.mean(residuals)
std_dev = np.std(residuals)

# Calculate the expected probabilities for each bin using the normal distribution
expected_probabilities = norm.pdf(bin_centers, loc=mean, scale=std_dev)

# Calculate the expected frequencies by multiplying the probability density by the total number of samples
expected_freq = expected_probabilities * np.sum(hist) * (bin_edges[1] - bin_edges[0])  # Multiply by bin width


Implement manual Chi-Squared calculation. I am doing some non robust stuff so the errors in scipy got to me

In [104]:
def chi_squared(observed, expected):
    # Ensure observed and expected have the same length
    if len(observed) != len(expected):
        raise ValueError("Observed and expected frequencies must have the same length.")

    # Calculate the Chi-Squared statistic
    chi2_stat = np.sum((observed - expected) ** 2 / expected)
    return chi2_stat

In [105]:
chi2_stat = chi_squared(hist, expected_freq)

print("Chi-Squared Statistic:", chi2_stat)

Chi-Squared Statistic: 16.7244414950924


This is our target to improve on. 

Our goal now is to iterate on the slope parameter and try to lower the chi squared statistic

In [106]:
def calculate_residuals(x, y, a, b):
    """
    Calculate residuals for a linear model y = ax + b.

    Parameters:
    x (numpy.ndarray): The input values.
    y (numpy.ndarray): The observed values.
    a (float): The slope of the line (coefficient).
    b (float): The intercept of the line.

    Returns:
    numpy.ndarray: The residuals (differences between observed and predicted values).
    """
    # Calculate predicted values using the model y = ax + b
    y_pred = a * x + b

    # Calculate residuals (observed - predicted)
    residuals = y - y_pred
    return residuals

In [107]:
def get_chi_score(x, y, a_est,b_est):
    # Calculate residuals
    residuals = calculate_residuals(x, y, a_est, b_est)

    # Step 5: Define the number of bins and compute the histogram (observed frequencies)
    num_bins = 10  # Example: 10 bins for the residuals
    hist, bin_edges = np.histogram(residuals, bins=num_bins, density=False)

    # Step 6: Calculate expected frequencies assuming a normal distribution
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])  # Compute bin centers
    mean = np.mean(residuals)
    std_dev = np.std(residuals)

    # Calculate the expected probabilities for each bin using the normal distribution
    expected_probabilities = norm.pdf(bin_centers, loc=mean, scale=std_dev)

    # Calculate the expected frequencies by multiplying the probability density by the total number of samples
    expected_freq = expected_probabilities * np.sum(hist) * (bin_edges[1] - bin_edges[0])  # Multiply by bin width

    # Step 8: Calculate Chi-Squared statistic manually
    chi2_stat = chi_squared(hist, expected_freq)

    return chi2_stat


you can play around with this section. eventually you find that the slope estimate near 2 gives the best fit

you will also find that the intercept estimate doesnt affect the residue fits. this is because it shifts the residues but not the goodness of fit to a normal distribution.

In [108]:
#1.86206803, 16.7244414950924
a_est = 1.97
b_est = 5

chi2_stat = get_chi_score(x, y, a_est, b_est)

# Print results
print("Chi-Squared Statistic:", chi2_stat)

Chi-Squared Statistic: 3.1883188240427156


this script runs through a list of possible values. if you perfer it that way

In [109]:
a_list = [slope + i * 0.01 for i in range(0, 25)]

In [110]:
chi_squared_dict = {}

for aentry in a_list:
    aentry = float(aentry)
    chi2_stat = get_chi_score(x, y, aentry, 5)
    chi_squared_dict[aentry] = chi2_stat

for key,value in chi_squared_dict.items():
    print(key,value)

1.8620680318630898 16.7244414950924
1.8720680318630898 17.12764892499672
1.8820680318630898 17.540554650041823
1.8920680318630898 17.57375436793365
1.9020680318630898 17.987464444947648
1.9120680318630898 18.774127643463096
1.9220680318630898 11.977207654568872
1.9320680318630898 12.711304068035416
1.9420680318630898 5.5280864276346104
1.9520680318630899 3.425009326694732
1.9620680318630899 3.454837250364978
1.9720680318630899 3.1960807237687403
1.9820680318630899 4.517569421352137
1.9920680318630897 4.570277694750394
2.00206803186309 4.624806652590812
2.0120680318630897 4.680841367288707
2.02206803186309 4.788019763788638
2.0320680318630897 4.839522871451056
2.04206803186309 4.278738027430414
2.0520680318630897 5.036641201168412
2.06206803186309 4.433445177075175
2.0720680318630897 4.5397216495540595
2.08206803186309 6.997169109919353
2.0920680318630898 7.362653220208978
2.10206803186309 6.335956470101273


C:\Users\andre\AppData\Local\Temp\ipykernel_10596\2131223492.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  aentry = float(aentry)


to deal with the intercept, we just force the slope and see what the mean of the intercept is

In [111]:
def calculate_intercept(x, y, m):
    # Calculate the intercept using the formula
    intercept = np.mean(y - m * x)
    return intercept

In [112]:
m_fixed = 1.972

intercept_calc = calculate_intercept(x, y, m_fixed)
print("Calculated intercept:", intercept)

Calculated intercept: [5.64528847]


Now its time to implement our test. set the samples to a ridiculously high number and see the MSE

In [113]:
n_samples = 100000  
x_eval  = np.random.rand(n_samples, 1) * 10  # Generate random x values between 0 and 10
noise_eval  = np.random.randn(n_samples, 1)  # Gaussian noise
y_eval = 2 * x_eval + 5 + noise_eval   # Linear function with Gaussian noise

y_norm = slope * x_eval + intercept
# Calculate mean squared error
mse = np.mean((y_eval - y_norm) ** 2)
print(f'Mean Squared Error: {mse}')

y_norm2 = m_fixed * x_eval + intercept_calc
# Calculate mean squared error
mse2 = np.mean((y_eval - y_norm2) ** 2)
print(f'Mean Squared Error: {mse2}')

Mean Squared Error: 1.1600652188277227
Mean Squared Error: 1.0041512398921222


Not bad, we even improved on the MSE!

The downside is scaling this up beyond 1 dimension. 